In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

print("Currently working at {}".format(os.getcwd()))

Currently working at /Users/gieunkwak/Data Analytics/random_things/tridge


In [2]:
import mysql.connector
from mysql.connector import Error
from pandas.io import sql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import time

import investpy
from fredapi import Fred
import json
from matplotlib import pyplot as plt
import quandl
from datetime import date
from app.utils.fetch_data import *
from app.utils.db_connection import *

In [8]:
results = fred.search('Producer Price Index by Commodity: Farm Products').reset_index()

In [9]:
results

,series id,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,WPU01190102,WPU01190102,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1991-12-01,2022-04-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Not Seasonally Adjusted,NSA,2022-05-12 07:52:06-05:00,46,None
1,WPS012202,WPS012202,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1975-01-01,2022-04-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2022-05-12 07:53:04-05:00,37,None
2,WPU012202,WPU012202,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1971-01-01,2022-04-01,Monthly,M,Index 1982=100,Index 1982=100,Not Seasonally Adjusted,NSA,2022-05-12 07:52:15-05:00,23,None
3,WPU01220205,WPU01220205,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1947-01-01,2022-04-01,Monthly,M,Index 1982=100,Index 1982=100,Not Seasonally Adjusted,NSA,2022-05-12 07:53:35-05:00,6,None
4,WPU01130603,WPU01130603,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1991-12-01,2022-04-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Not Seasonally Adjusted,NSA,2022-05-12 07:52:07-05:00,38,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,WPS0143,WPS0143,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1993-01-01,1997-12-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Seasonally Adjusted,SA,2015-05-14 10:44:53-05:00,1,None
241,WPSSOP1400,WPSSOP1400,2022-05-16,2022-05-16,Producer Price Index by Commodity for Stage of...,1947-04-01,2015-12-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2016-01-15 09:01:09-06:00,1,"In January 2014, the Producer Price Index (PPI..."
242,WPUSOP1400,WPUSOP1400,2022-05-16,2022-05-16,Producer Price Index by Commodity for Stage of...,1947-01-01,2015-12-01,Monthly,M,Index 1982=100,Index 1982=100,Not Seasonally Adjusted,NSA,2016-01-15 08:55:05-06:00,1,"In January 2014, the Producer Price Index (PPI..."
243,WPS0653,WPS0653,2022-05-16,2022-05-16,Producer Price Index by Commodity: Chemicals a...,1976-01-01,1994-12-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2015-05-14 10:42:35-05:00,1,None


In [31]:
with open('app/keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)

## Create database and table

In [22]:
connection = create_server_connection(keys['mysql']['host_name'], keys['mysql']['user_name'], 
                                      keys['mysql']['user_password'])

2022-05-16 20:37:59,171 [app.utils.db_connection] [INFO] MySQL Database connection successful


In [23]:
create_database(connection, "CREATE DATABASE food")

2022-05-16 20:38:08,973 [app.utils.db_connection] [INFO] Database created successfully


In [26]:
connection = create_db_connection(keys['mysql']['host_name'], keys['mysql']['user_name'], 
                                  keys['mysql']['user_password'], 'food')

2022-05-16 20:39:13,727 [app.utils.db_connection] [INFO] MySQL Database connection successful


In [29]:
execute_query(connection, 'DROP TABLE fred_index') # delete table

create_fred_table = """
CREATE TABLE fred_index (
    p_key VARCHAR(50) PRIMARY KEY,
    date DATE,
    v FLOAT(50, 2),
    code VARCHAR(50) NOT NULL
    );
"""

execute_query(connection, create_fred_table)

2022-05-16 20:39:31,314 [app.utils.db_connection] [INFO] Query successful
2022-05-16 20:39:31,323 [app.utils.db_connection] [INFO] Query successful


In [30]:
connection.close()

## Collect data and populate

In [32]:
# mysql+pymysql://[db유저이름]:[db password]@[host address]/[db name]

db_connection_str = 'mysql+pymysql://' + keys['mysql']['user_name'] + ":" + keys['mysql']['user_password'] + \
                    "@" + "127.0.0.1:" + keys['mysql']['host_address'] + '/food'
# root:gi941505@127.0.0.1:3306/econ'
db_connection = create_engine(db_connection_str)
conn = db_connection.raw_connection()

In [33]:
results

,series id,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,WPU01190102,WPU01190102,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1991-12-01,2022-04-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Not Seasonally Adjusted,NSA,2022-05-12 07:52:06-05:00,46,None
1,WPS012202,WPS012202,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1975-01-01,2022-04-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2022-05-12 07:53:04-05:00,37,None
2,WPU012202,WPU012202,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1971-01-01,2022-04-01,Monthly,M,Index 1982=100,Index 1982=100,Not Seasonally Adjusted,NSA,2022-05-12 07:52:15-05:00,23,None
3,WPU01220205,WPU01220205,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1947-01-01,2022-04-01,Monthly,M,Index 1982=100,Index 1982=100,Not Seasonally Adjusted,NSA,2022-05-12 07:53:35-05:00,6,None
4,WPU01130603,WPU01130603,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1991-12-01,2022-04-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Not Seasonally Adjusted,NSA,2022-05-12 07:52:07-05:00,38,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,WPS0143,WPS0143,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1993-01-01,1997-12-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Seasonally Adjusted,SA,2015-05-14 10:44:53-05:00,1,None
241,WPSSOP1400,WPSSOP1400,2022-05-16,2022-05-16,Producer Price Index by Commodity for Stage of...,1947-04-01,2015-12-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2016-01-15 09:01:09-06:00,1,"In January 2014, the Producer Price Index (PPI..."
242,WPUSOP1400,WPUSOP1400,2022-05-16,2022-05-16,Producer Price Index by Commodity for Stage of...,1947-01-01,2015-12-01,Monthly,M,Index 1982=100,Index 1982=100,Not Seasonally Adjusted,NSA,2016-01-15 08:55:05-06:00,1,"In January 2014, the Producer Price Index (PPI..."
243,WPS0653,WPS0653,2022-05-16,2022-05-16,Producer Price Index by Commodity: Chemicals a...,1976-01-01,1994-12-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2015-05-14 10:42:35-05:00,1,None


In [34]:
today = date.today().strftime("%Y-%m-%d")
today

'2022-05-16'

In [35]:
df = fred_fred('WPU01190102', observation_start='2000-01-01', observation_end=today)

2022-05-16 20:44:08,032 [app.utils.fetch_data] [INFO] Fetching data from fred: WPU01190102, from 2000-01-01 to 2022-05-16.


In [37]:
df.to_sql(name='fred_index', con=db_connection, if_exists='append', index=False)

In [38]:
conn.close()